In [34]:
#Wyatt Kirby
#IRK180000
#CS 4395.001
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#importing what is needed for Naive Bayes
import numpy as np
import pandas as pd

df = pd.read_csv('/kaggle/input/mental-health-corpus/mental_health.csv', encoding='utf-8')
print(df.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

                                                text  label
0  dear american teens question dutch person hear...      0
1  nothing look forward lifei dont many reasons k...      1
2  music recommendations im looking expand playli...      0
3  im done trying feel betterthe reason im still ...      1
4  worried  year old girl subject domestic physic...      1


With this dataset I should be able to classify some text as possibly being indicative of mental health issues or not. In the 'label' column, a 0 indicates that there no need to worry.

In [35]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

#Removing the stop words from the data
stopwords = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(stop_words=stopwords)

#setting up X and Y
x = df.text
y = df.label

In [36]:
x.head()

0    dear american teens question dutch person hear...
1    nothing look forward lifei dont many reasons k...
2    music recommendations im looking expand playli...
3    im done trying feel betterthe reason im still ...
4    worried  year old girl subject domestic physic...
Name: text, dtype: object

In [37]:
y[:5]

0    0
1    1
2    0
3    1
4    1
Name: label, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=.2, train_size=.8, random_state=1234)

print(x.shape)
print(xtrain.shape)
print(xtest.shape)

(27977,)
(22381,)
(5596,)


As you can see the training set has 80% of the initial dataset and the testing sets contain 20%

In [39]:
xtrain = vectorizer.fit_transform(xtrain)
xtest = vectorizer.transform(xtest)   

In [40]:
print('train size:', xtrain.shape)
print(xtrain.toarray()[:5])

print('\ntest size:', xtest.shape)
print(xtest.toarray()[:5])

train size: (22381, 63944)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

test size: (5596, 63944)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**Naive Bayes:**

In [41]:
from sklearn.naive_bayes import MultinomialNB

bayes = MultinomialNB()
bayes.fit(xtrain, ytrain)

MultinomialNB()

In [42]:
import math
prior = sum(ytrain==1)/len(ytrain)
print('prior label:', prior, 'log of prior:', math.log(prior))
bayes.class_log_prior_[1]

prior label: 0.4930521424422501 log of prior: -0.7071403449292926


-0.707140344929293

In [43]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

pred = bayes.predict(xtest)

print(confusion_matrix(ytest, pred))

[[2004  789]
 [  31 2772]]


As you can see, there were far more true negative and true positives than there were msitakes, meaning that Naive Bayes was farily accurate without needing more fine tuning.

In [44]:
print('accuracy score: ', accuracy_score(ytest, pred))
      
print('\nprecision score (no sign): ', precision_score(ytest, pred, pos_label=0))
print('precision score (sign): ', precision_score(ytest, pred))

print('\nrecall score: (not sign)', recall_score(ytest, pred, pos_label=0))
print('recall score: (sign)', recall_score(ytest, pred))
      
print('\nf1 score: ', f1_score(ytest, pred))

accuracy score:  0.8534667619728378

precision score (no sign):  0.9847665847665847
precision score (sign):  0.7784330244313395

recall score: (not sign) 0.7175080558539205
recall score: (sign) 0.9889404209775241

f1 score:  0.8711502199874293


Naive Bayes had a accuracy score of 85%, is more precise at identifying text that does not show signs of mental illness, and has a better recall for text that does show sign of mental illness.

In [45]:
from sklearn.metrics import classification_report
print(classification_report(ytest, pred))

              precision    recall  f1-score   support

           0       0.98      0.72      0.83      2793
           1       0.78      0.99      0.87      2803

    accuracy                           0.85      5596
   macro avg       0.88      0.85      0.85      5596
weighted avg       0.88      0.85      0.85      5596



**Logistic Regression:**

In [46]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', class_weight='balanced')
classifier.fit(xtrain, ytrain)

LogisticRegression(class_weight='balanced')

In [47]:
pred = classifier.predict(xtest)
print(confusion_matrix(ytest, pred))

[[2604  189]
 [ 281 2522]]


This was was more accurate than the Naive Bayes model, as it identified more true positive and true negative. Had I tuned Bayes adn tried again it might have peformed similarly, but I think it's better to just run all of them once and compare how they all did initially.

In [48]:
print('accuracy score: ', accuracy_score(ytest, pred))
      
print('\nprecision score (no sign): ', precision_score(ytest, pred, pos_label=0))
print('precision score (sign): ', precision_score(ytest, pred))

print('\nrecall score: (not sign)', recall_score(ytest, pred, pos_label=0))
print('recall score: (sign)', recall_score(ytest, pred))
      
print('\nf1 score: ', f1_score(ytest, pred))

accuracy score:  0.916011436740529

precision score (no sign):  0.9025996533795494
precision score (sign):  0.9302840280339358

recall score: (not sign) 0.9323308270676691
recall score: (sign) 0.8997502675704602

f1 score:  0.9147624229234675


As we can see here, the accuracy of Logistic Regression is 91%. All scored for logistic regression surpass those from the Bayes model.

In [49]:
print(classification_report(ytest, pred))

              precision    recall  f1-score   support

           0       0.90      0.93      0.92      2793
           1       0.93      0.90      0.91      2803

    accuracy                           0.92      5596
   macro avg       0.92      0.92      0.92      5596
weighted avg       0.92      0.92      0.92      5596



**Neural Network:**

In [50]:
from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15, 2), random_state=1)
classifier.fit(xtrain, ytrain)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15, 2), random_state=1,
              solver='lbfgs')

In [51]:
pred = classifier.predict(xtest)
print(confusion_matrix(ytest, pred))

[[2793    0]
 [2803    0]]


It seems as though the neural network only guessed that each text has no signs of mental illness. Lets check.

In [52]:
pred[:5]

array([0, 0, 0, 0, 0])

As we can see, this is correct for at least the first five. I mostly did this just to check which one the Neural Network was guessing.

In [53]:
print('accuracy score: ', accuracy_score(ytest, pred))
print('precision score: ', precision_score(ytest, pred, pos_label=0))
print('recall score: ', recall_score(ytest, pred, pos_label=0))
print('f1 score: ', f1_score(ytest, pred, pos_label=0))

accuracy score:  0.49910650464617584
precision score:  0.49910650464617584
recall score:  1.0
f1 score:  0.6658719752056265


This one had such low scores for accuracy because it only chose that the text showed no sign of mental illness. It really had 50 / 50 odds as there were only two options and the dataset seems to have been evenly split between the two. This means that the precision, recall, and f1 scores mean basically next to nothing.

# Analysis
**Naive Bayes**

The naive bayes model performed the second best between all three. This model used a fairly small dataset and only had two options, making it a slightly easier model for me to train on. This being said, this model had a fairly high accuracy score of 85%. It also has a incredibly high precision score (91%) for predicting the text without worrying language, meaning that it made very few errors and was better at predicting those. I am unsure why there was such a drop with the other category (roughly 78%). This could be due to some grammatical errors, punctuation, or spelling errors that those in an unstable frame of mind might be more likely to make than someone who is feeling fine in that moment. These errors could throw off the model and make it harder to predict for those catagories. The recall was the opposite of the precision, having a higher percentage with the worrying text (99%) than the healthy text samples (78%). This may be due to the issues that caused a lower precision, as a person in crisis may have a more distincive writing style than that of someone who is fine. This would mean that out of all the text that is worrying, it would be easier to identify those, but might also include text that isn't worrying from someone who may not type well or have developed their own writing style when texting friends.

**Logistic Regression**

This model performed the best out of all of the models in accuracy and had a more even distribution with in precision and recall. The accuracy was a whopping 92%, which is incredibly impressive considering that this is the initial run with no fine tuning. All of the scores for recall and precision were in the 90th percentile, so that means that not only was it likely that if it made a guess, that that guess was correct and that overall all the majority of the text was categorized correctly. I think that the distinctive writing styles that may have confused the Bayes model helped Logistic Regression make a more informed decision when catagorizing everything.

**Neural Networks**

This one performed so poorly. Everything was very low. I tried tinkering with various input values, but it didn't make much of an impact at all (usually causing the performance overall to deteriorate further). The overall accuracy was at 50%, and looking at the confusion matrix its clear to see that the model only guessed that the text was normal and healthy. This means that everything was split at the middle, as we see with the accuracy and precision. Recall was high only because it guessed that all of the healthy text was healthy by default. The recall for the worrying messages was at zero. Overall dissapointing. In the machine learning class I took last year the Neural Networks seemed to perform better than the other algorithms, but maybe it struggles when spelling errors and grammatical errors are present.